In [64]:
%matplotlib inline

In [65]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [66]:
import utils; reload(utils)
from utils import plots

In [92]:
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [93]:
path = "data/dogscats/"
#path = "data/dogscats/sample/"
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=90
epochs = 1

In [94]:
vgg = Vgg16()
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
batches.class_indices
val_batches.class_indices

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


{'cats': 0, 'dogs': 1}

In [95]:
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=epochs)

Epoch 1/1
23000/23000 [==============================] - 620s - loss: 0.1285 - acc: 0.9656 - val_loss: 0.0559 - val_acc: 0.9830


In [99]:
#Adding transformations
epochs = 5
from keras.preprocessing import image
train_transformed = vgg.get_batches(path+"train", batch_size=batch_size, gen=image.ImageDataGenerator(shear_range=0.1,rotation_range=5,vertical_flip=True,width_shift_range=0.02, height_shift_range=0.02 ))
vgg.fit(train_transformed, val_batches, nb_epoch=epochs)

Found 23000 images belonging to 2 classes.
Epoch 1/5
23000/23000 [==============================] - 623s - loss: 0.3319 - acc: 0.9119 - val_loss: 0.0889 - val_acc: 0.9740
Epoch 2/5
23000/23000 [==============================] - 622s - loss: 0.2836 - acc: 0.9197 - val_loss: 0.0831 - val_acc: 0.9740
Epoch 3/5
23000/23000 [==============================] - 621s - loss: 0.2668 - acc: 0.9226 - val_loss: 0.0587 - val_acc: 0.9795
Epoch 4/5
23000/23000 [==============================] - 621s - loss: 0.2640 - acc: 0.9236 - val_loss: 0.0598 - val_acc: 0.9795
Epoch 5/5
23000/23000 [==============================] - 622s - loss: 0.2765 - acc: 0.9198 - val_loss: 0.0767 - val_acc: 0.9815


In [80]:
train_transformed = vgg.get_batches(path+"train", batch_size=batch_size, gen=image.ImageDataGenerator(rotation_range=5))
vgg.fit(train_transformed, val_batches, nb_epoch=epochs)

Found 23000 images belonging to 2 classes.
Epoch 1/1
23000/23000 [==============================] - 611s - loss: 0.1641 - acc: 0.9642 - val_loss: 0.0499 - val_acc: 0.9850


In [83]:
train_transformed = vgg.get_batches(path+"train", batch_size=batch_size, gen=image.ImageDataGenerator(vertical_flip=True))
vgg.fit(train_transformed, val_batches, nb_epoch=epochs)

Found 23000 images belonging to 2 classes.
Epoch 1/1
23000/23000 [==============================] - 613s - loss: 0.4193 - acc: 0.9169 - val_loss: 0.0855 - val_acc: 0.9805


In [86]:
train_transformed = vgg.get_batches(path+"train", batch_size=batch_size, gen=image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1))
vgg.fit(train_transformed, val_batches, nb_epoch=epochs)

Found 23000 images belonging to 2 classes.
Epoch 1/1
23000/23000 [==============================] - 611s - loss: 0.1130 - acc: 0.9745 - val_loss: 0.0831 - val_acc: 0.9790


# ---------------------------------------------------------

In [101]:
test_batches, predictions = vgg.test(path+'test', batch_size)

Found 12500 images belonging to 1 classes.


In [77]:
print(test_batches.filenames[0:10])
print(predictions.shape)

['fakeclass/9292.jpg', 'fakeclass/12026.jpg', 'fakeclass/9688.jpg', 'fakeclass/4392.jpg', 'fakeclass/779.jpg', 'fakeclass/2768.jpg', 'fakeclass/2399.jpg', 'fakeclass/12225.jpg', 'fakeclass/10947.jpg', 'fakeclass/1780.jpg']
(12500, 2)


In [102]:
#Return image ID from its name
def get_id(img_path):
    fid = img_path.split("/")[1].split(".")[0] #Get image ID
    return fid

ids = []
dog_probs = []
for img_path,pred in zip(test_batches.filenames, predictions):
    fid = get_id(img_path)
    dog_prob = pred[1]
    ids.append(fid)
    dog_probs.append(dog_prob)
    
df = pd.DataFrame({"id":ids, "label":dog_probs})
#print(df)
fn = "./sub/train2_batchsize" + str(batch_size) + "_epochs" + str(epochs) +"_all_transforms.csv"
df.to_csv(fn, index=False)
print("Write to " + fn)
print("DONE!")
from IPython.display import FileLink
FileLink(fn)  

Write to ./sub/train2_batchsize90_epochs5_all_transforms.csv
DONE!


/home/ubuntu/nbs/ml/sub/train2_batchsize90_epochs5_all_transforms.csv

#Training History
Train Session 1:
1. Base line train: re-pull code from github, re-wrote the code to run the deep learning, used default batch_size = 64, epochs =1
    Epoch 1/1
23000/23000 [==============================] - 612s - loss: 0.1197 - acc: 0.9684 - val_loss: 0.0665 - val_acc: 0.9830
Test score: 0.17580, filename: batchsize64_epochs1_base.csv
2. Trained based on the base line model, added shear_range = 0.1
Epoch 1/1
23000/23000 [==============================] - 613s - loss: 0.1014 - acc: 0.9756 - val_loss: 0.0667 - val_acc: 0.9810
Test score: 0.16436, filename: batchsize64_epochs1_base_shear.csv    
3. base + shear + rotate (30)
Epoch 1/1
23000/23000 [==============================] - 611s - loss: 0.1641 - acc: 0.9642 - val_loss: 0.0499 - val_acc: 0.9850
Test score: 0.17411, filename: batchsize64_epochs1_base_shear_rotate.csv  
4. base + shear + rotate + vertical flip
23000/23000 [==============================] - 613s - loss: 0.4193 - acc: 0.9169 - val_loss: 0.0855 - val_acc: 0.9805
Test score: 0.16267, filename: batchsize64_epochs1_base_shear_rotate_flip.csv  
5. base + shear + rotate + flip + shift (width+height:0.1)
23000/23000 [==============================] - 611s - loss: 0.1130 - acc: 0.9745 - val_loss: 0.0831 - val_acc: 0.9790
Test score: 0.17426, filename: batchsize64_epochs1_base_shear_rotate_flip_shift.csv  
6. changed batch_size = 90, retrain the raw images based on the model from step 1-5
23000/23000 [==============================] - 613s - loss: 0.0936 - acc: 0.9794 - val_loss: 0.0754 - val_acc: 0.9825
Test score: 0.19362, filename: batchsize90_epochs1_base_shear_rotate_flip_shift.csv

Train Session 2:        
1. batch_size = 90, epochs = 1, start afresh
23000/23000 [==============================] - 620s - loss: 0.1285 - acc: 0.9656 - val_loss: 0.0559 - val_acc: 0.9830
Test score: 0.13399, filename: train2_batchsize90_epochs1.csv
2. batch_size = 90, epochs = 5, added transformations       
train_transformed = vgg.get_batches(path+"train", batch_size=batch_size, gen=image.ImageDataGenerator(shear_range=0.1,rotation_range=5,vertical_flip=True,width_shift_range=0.02, height_shift_range=0.02 ))

Epoch 1/5
23000/23000 [==============================] - 623s - loss: 0.3319 - acc: 0.9119 - val_loss: 0.0889 - val_acc: 0.9740
Epoch 2/5
23000/23000 [==============================] - 622s - loss: 0.2836 - acc: 0.9197 - val_loss: 0.0831 - val_acc: 0.9740
Epoch 3/5
23000/23000 [==============================] - 621s - loss: 0.2668 - acc: 0.9226 - val_loss: 0.0587 - val_acc: 0.9795
Epoch 4/5
23000/23000 [==============================] - 621s - loss: 0.2640 - acc: 0.9236 - val_loss: 0.0598 - val_acc: 0.9795
Epoch 5/5
23000/23000 [==============================] - 622s - loss: 0.2765 - acc: 0.9198 - val_loss: 0.0767 - val_acc: 0.9815
In [80]:
Test score: 0.14027, filename:train2_batchsize90_epochs5_all_transforms.csv